In [1]:
# 导入需要的包, Sequential()是最简单的模型——序贯模型
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
import cv2


Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(Dense(500,input_shape=(3072,))) #输入层， 28*28=784
model.add(Activation('tanh'))
model.add(Dropout(0.5)) #50% dropout

model.add(Dense(500)) #隐藏层， 500
model.add(Activation('tanh'))
model.add(Dropout(0.5)) #50% dropout

model.add(Dense(8)) #输出结果， 10
model.add(Activation('softmax'))

In [3]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) #设定学习效率等参数
model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) #使用交叉熵作为loss

In [4]:
def get_data():
    X_t1=[]
    X_t2=[]
    Y_t1=[]
    Y_t2=[]
    for i in range(8):
        img=cv2.imread(('data/test'+str(i)+'.png'))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        X_t1.append(gray)
        Y_t1.append(i)
        if i==7:
            X_t2.append(gray)
            Y_t2.append(i)
    a=(np.array(X_t1),np.array(Y_t1))
    b=(np.array(X_t2),np.array(Y_t2))
    return a,b
            
        
        
    

In [5]:
(x_train,y_train),(x_test,y_test) = get_data() #使用mnist读取数据（第一次需要下载）

print(y_train)

X_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
X_test = x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2])

print(X_train.shape)
Y_train = (np.arange(8) == y_train[:,None]).astype(int) #将index转换橙一个one_hot矩阵
Y_test = (np.arange(8) == y_test[:,None]).astype(int)
print (Y_train.shape)

[0 1 2 3 4 5 6 7]
(8, 3072)
(8, 8)


In [6]:
model.fit(X_train,Y_train,batch_size=2,epochs=8,shuffle=True,verbose=1,validation_split=0.3)

Train on 5 samples, validate on 3 samples
Epoch 1/8
5/5 [==============================] - 2s 324ms/step - loss: 5.1345 - acc: 0.0000e+00 - val_loss: 9.5557 - val_acc: 0.0000e+00
Epoch 2/8
5/5 [==============================] - 0s 11ms/step - loss: 5.3884 - acc: 0.0000e+00 - val_loss: 12.7167 - val_acc: 0.0000e+00
Epoch 3/8
5/5 [==============================] - 0s 10ms/step - loss: 6.4736 - acc: 0.0000e+00 - val_loss: 11.8539 - val_acc: 0.0000e+00
Epoch 4/8
5/5 [==============================] - 0s 10ms/step - loss: 5.5157 - acc: 0.2000 - val_loss: 13.5394 - val_acc: 0.0000e+00
Epoch 5/8
5/5 [==============================] - 0s 10ms/step - loss: 7.3387 - acc: 0.2000 - val_loss: 10.1811 - val_acc: 0.0000e+00
Epoch 6/8
5/5 [==============================] - 0s 10ms/step - loss: 6.6829 - acc: 0.4000 - val_loss: 9.8685 - val_acc: 0.0000e+00
Epoch 7/8
5/5 [==============================] - 0s 9ms/step - loss: 5.8521 - acc: 0.4000 - val_loss: 11.3089 - val_acc: 0.0000e+00
Epoch 8/8
5/5 [==

In [8]:
# 输出对测试集进行测试的结果
print("test set")
np.set_printoptions(threshold=90000)

X_test1=X_train
#print(X_test1)
Y_test1=Y_train
print(Y_test1)
scores = model.evaluate(X_test1,Y_test1,batch_size=2,verbose=1)
print("")
print("The test loss is")
print(scores)
result = model.predict(X_test1,batch_size=2,verbose=1)
print("The redult is")
print (result)

result_max = np.argmax(result, axis = 1)
test_max = np.argmax(Y_test1, axis = 1)
#print(result)

result_bool = np.equal(result_max, test_max)

true_num = np.sum(result_bool)
print("true_num=",true_num)
print("")
print("The accuracy of the model is %f" % (true_num/len(result_bool)))
# 详解
# model.evaluate()计算了测试集中的识别的loss值。
# 通过model.predict()，我们可以得到对于测试集中每个数字的识别结果，每个数字对应一个表示每个数字都是多少概率的长度为10的张量。
# 通过np.argmax()，我们得到每个数字的识别结果和期望的识别结果
# 通过np.equal()，我们得到每个数字是否识别正确
# 通过np.sum()得到识别正确的总的数字个数

# 作者：JackieXiao
# 链接：https://www.jianshu.com/p/6bf2c54a9d60
# 來源：简书
# 简书著作权归作者所有，任何形式的转载都请联系作者获得授权并注明出处。

test set
[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]
8/8 [==============================] - 0s 1ms/step

The test loss is
[8.149681329727173, 0.125]
8/8 [==============================] - 0s 1ms/step
The redult is
[[3.2853932e-05 3.8656758e-04 8.6795082e-03 7.8067387e-06 9.9086809e-01
  2.1586713e-05 1.4348946e-07 3.4295460e-06]
 [3.0585063e-05 2.3824377e-03 4.0433101e-02 6.1519124e-05 9.5704895e-01
  3.5891899e-05 3.5718500e-07 7.0504079e-06]
 [2.7170227e-05 1.4546184e-03 2.6352167e-01 3.3179738e-05 7.3488325e-01
  6.4231412e-05 6.7440584e-07 1.5176413e-05]
 [6.8522727e-06 2.5042819e-04 3.1267379e-03 1.4090186e-04 9.9645293e-01
  1.8493940e-05 1.4111548e-07 3.5970018e-06]
 [5.6327390e-06 1.4412738e-04 8.8003289e-04 3.2009373e-06 9.9896133e-01
  4.3966952e-06 4.5340077e-08 1.3160922e-06]
 [2.8259601e-05 8.6522102e-04 3.0759893e-02 2.2303842e-05 9.6828622e-01
  3.1644908e-05 2.66